1.Let’s attempt to predict the survival of a horse based on various observed medical conditions. Load the data from ‘horses.csv’ and observe whether it contains missing values.[Hint: Pandas dataframe has a method isnull]

In [270]:
import pandas as pd
horseD=pd.read_csv('horse.csv')

In [271]:
horseD.dtypes

surgery                   object
age                       object
hospital_number            int64
rectal_temp              float64
pulse                    float64
respiratory_rate         float64
temp_of_extremities       object
peripheral_pulse          object
mucous_membrane           object
capillary_refill_time     object
pain                      object
peristalsis               object
abdominal_distention      object
nasogastric_tube          object
nasogastric_reflux        object
nasogastric_reflux_ph    float64
rectal_exam_feces         object
abdomen                   object
packed_cell_volume       float64
total_protein            float64
abdomo_appearance         object
abdomo_protein           float64
outcome                   object
surgical_lesion           object
lesion_1                   int64
lesion_2                   int64
lesion_3                   int64
cp_data                   object
dtype: object

In [272]:
horseD.isna().sum()

surgery                    0
age                        0
hospital_number            0
rectal_temp               60
pulse                     24
respiratory_rate          58
temp_of_extremities       56
peripheral_pulse          69
mucous_membrane           47
capillary_refill_time     32
pain                      55
peristalsis               44
abdominal_distention      56
nasogastric_tube         104
nasogastric_reflux       106
nasogastric_reflux_ph    246
rectal_exam_feces        102
abdomen                  118
packed_cell_volume        29
total_protein             33
abdomo_appearance        165
abdomo_protein           198
outcome                    0
surgical_lesion            0
lesion_1                   0
lesion_2                   0
lesion_3                   0
cp_data                    0
dtype: int64

In [273]:
z=list(horseD.dtypes==object)

In [274]:
categorical_col=horseD.dtypes[z].index

In [275]:
nn=horseD.dtypes[horseD.isna().sum()>0].index
nn #col having nan continuous features

Index(['rectal_temp', 'pulse', 'respiratory_rate', 'temp_of_extremities',
       'peripheral_pulse', 'mucous_membrane', 'capillary_refill_time', 'pain',
       'peristalsis', 'abdominal_distention', 'nasogastric_tube',
       'nasogastric_reflux', 'nasogastric_reflux_ph', 'rectal_exam_feces',
       'abdomen', 'packed_cell_volume', 'total_protein', 'abdomo_appearance',
       'abdomo_protein'],
      dtype='object')

2.This dataset contains many categorical features, replace them with label encoding.[Hint: Refer to get_dummies methods in pandas dataframe or Label encoder in scikit-learn]
3.Replace the missing values by the most frequent value in each column.[Hint: Refer to Imputer class in Scikit learn preprocessing module]

In [276]:
#categorical_col.values
horseD.loc[:, categorical_col.values].head(5)

,surgery,age,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,rectal_exam_feces,abdomen,abdomo_appearance,outcome,surgical_lesion,cp_data
0,no,adult,cool,reduced,NaN,more_3_sec,extreme_pain,absent,severe,NaN,NaN,decreased,distend_large,NaN,died,no,no
1,yes,adult,NaN,NaN,pale_cyanotic,less_3_sec,mild_pain,absent,slight,NaN,NaN,absent,other,cloudy,euthanized,no,no
2,no,adult,normal,normal,pale_pink,less_3_sec,mild_pain,hypomotile,none,NaN,NaN,normal,normal,NaN,lived,no,yes
3,yes,young,cold,normal,dark_cyanotic,more_3_sec,depressed,absent,severe,none,less_1_liter,decreased,NaN,serosanguious,died,yes,yes
4,no,adult,NaN,NaN,dark_cyanotic,more_3_sec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,died,no,no


In [277]:
#doc-Since the scikit-learn Imputer transformer currently only works with numbers, sklearn-pandas provides an equivalent helper transformer that do work with strings, substituting null values with the most frequent value in that column.
from sklearn_pandas import CategoricalImputer
imputer = CategoricalImputer()
#temp=imputer.fit_transform(horseD.loc[:, categorical_col.values[1]])

In [278]:
for i in categorical_col.values:
    horseD.loc[:, i]=imputer.fit_transform(horseD.loc[:, i])

In [279]:
horseD.loc[:, categorical_col.values].head(5)

,surgery,age,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,rectal_exam_feces,abdomen,abdomo_appearance,outcome,surgical_lesion,cp_data
0,no,adult,cool,reduced,normal_pink,more_3_sec,extreme_pain,absent,severe,slight,none,decreased,distend_large,cloudy,died,no,no
1,yes,adult,cool,normal,pale_cyanotic,less_3_sec,mild_pain,absent,slight,slight,none,absent,other,cloudy,euthanized,no,no
2,no,adult,normal,normal,pale_pink,less_3_sec,mild_pain,hypomotile,none,slight,none,normal,normal,cloudy,lived,no,yes
3,yes,young,cold,normal,dark_cyanotic,more_3_sec,depressed,absent,severe,none,less_1_liter,decreased,distend_large,serosanguious,died,yes,yes
4,no,adult,cool,normal,dark_cyanotic,more_3_sec,mild_pain,hypomotile,none,slight,none,absent,distend_large,cloudy,died,no,no


In [280]:
col=set(nn).difference(categorical_col)
col

{'abdomo_protein',
 'nasogastric_reflux_ph',
 'packed_cell_volume',
 'pulse',
 'rectal_temp',
 'respiratory_rate',
 'total_protein'}

In [281]:
from sklearn.impute import SimpleImputer 
imputer=SimpleImputer(missing_values=np.nan,strategy='most_frequent')
imputer=imputer.fit(horseD.loc[:, col])
horseD.loc[:, col]=imputer.transform(horseD.loc[:, col])


In [282]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [283]:
le.fit(horseD['cp_data'])
horseD['cp_data']=le.transform(horseD['cp_data'])

In [284]:
horseD2=pd.get_dummies(horseD)

In [285]:
horseD2.head()

,hospital_number,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,...,abdomen_normal,abdomen_other,abdomo_appearance_clear,abdomo_appearance_cloudy,abdomo_appearance_serosanguious,outcome_died,outcome_euthanized,outcome_lived,surgical_lesion_no,surgical_lesion_yes
0,530101,38.5,66.0,28.0,2.0,45.0,8.4,2.0,11300,0,...,0,0,0,1,0,1,0,0,1,0
1,534817,39.2,88.0,20.0,2.0,50.0,85.0,2.0,2208,0,...,0,1,0,1,0,0,1,0,1,0
2,530334,38.3,40.0,24.0,2.0,33.0,6.7,2.0,0,0,...,1,0,0,1,0,0,0,1,1,0
3,5290409,39.1,164.0,84.0,5.0,48.0,7.2,5.3,2208,0,...,0,0,0,0,1,1,0,0,0,1
4,530255,37.3,104.0,35.0,2.0,74.0,7.4,2.0,4300,0,...,0,0,0,1,0,1,0,0,1,0


4.Fit a decision tree classifier and observe the accuracy.
5.Fit a random forest classifier and observe the accuracy.

In [286]:
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics

In [287]:
train,test=train_test_split(horseD2,test_size=0.3)
print(train.shape)
print(test.shape)

(209, 69)
(90, 69)


In [288]:
train_x=train.iloc[:,:-1]
train_y=train['cp_data']
test_x=test.iloc[:,:-1]
test_y=test['cp_data']

In [289]:
model=tree.DecisionTreeClassifier()
model.fit(train_x,train_y)
y_pred=model.predict(test_x)
print(metrics.accuracy_score(y_pred,test_y))

from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=100)
model.fit(train_x,train_y)
y_pred=model.predict(test_x)
print(metrics.accuracy_score(y_pred,test_y))

1.0
0.9888888888888889


In [290]:
pd.DataFrame(y_pred,test_y).reset_index()

,cp_data,0
0,0,0
1,0,0
2,1,1
3,1,1
4,1,1
5,0,0
6,0,0
7,0,0
8,1,1
9,0,0
